<a href="https://colab.research.google.com/github/evpozdniakov/ds_projects/blob/master/hw2/mussels.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Загружаем необходимые библиотеки

In [92]:
import pandas as pd
from scipy.stats import shapiro
from scipy.stats import pearsonr
from scipy.stats import ttest_ind

Объявляем исходные данные: размеры раковин мидий выращенных в разных городах-производителях. (Их мы получили вместе с заданием. Также известно, что данные имеют *нормальное распределение*.)

In [93]:
petersburg = [0.0974, 0.1352, 0.0817, 0.1016, 0.0968, 0.1064, 0.105]
magadan = [0.1033, 0.0915, 0.0781, 0.0685, 0.0677, 0.0697, 0.0764, 0.0689]

Теперь можно приступать к работе. Выполним первый шаг.

#### 1. Объединить 2 массива в DataFrame

Наши массивы имеют разный размер. Мы можем воспользоваться стандартным методом создания датафрейма из Series:

```python
pd.DataFrame({'a': pd.Series(a), 'b': pd.Series(b)})
```

In [94]:
df = pd.DataFrame({
    'St.Petersburg': pd.Series(petersburg),
    'Magadan': pd.Series(magadan),
})

Проверяем результат

In [95]:
df.head(10)

,St.Petersburg,Magadan
0,0.0974,0.1033
1,0.1352,0.0915
2,0.0817,0.0781
3,0.1016,0.0685
4,0.0968,0.0677
5,0.1064,0.0697
6,0.1050,0.0764
7,NaN,0.0689


Мы видим, что недостающие данные были заменены на **NaN**. Прежде чем продолжать работу, нам нужно исправить эту ситуацию.

Недостающие данные мы можем заполнить или *средним* значением данного признака, или *медианным*. В нашем случае выбор не будет иметь решающего значения, но на всякий случай сделаем два варианта и проверим как они себя поведут, и повлияет ли этот выбор на конечный результат.

In [96]:
mask = df['St.Petersburg'].isna()

df_clean_avg = df.copy()
df_clean_avg.loc[mask, 'St.Petersburg'] = df['St.Petersburg'].mean()

df_clean_med = df.copy()
df_clean_med.loc[mask, 'St.Petersburg'] = df['St.Petersburg'].median()

Проверяем что получилось.

In [97]:
print(df_clean_avg, df_clean_med, sep='\n\n')

   St.Petersburg  Magadan
0       0.097400   0.1033
1       0.135200   0.0915
2       0.081700   0.0781
3       0.101600   0.0685
4       0.096800   0.0677
5       0.106400   0.0697
6       0.105000   0.0764
7       0.103443   0.0689

   St.Petersburg  Magadan
0         0.0974   0.1033
1         0.1352   0.0915
2         0.0817   0.0781
3         0.1016   0.0685
4         0.0968   0.0677
5         0.1064   0.0697
6         0.1050   0.0764
7         0.1016   0.0689


Далее в нашем исследовании мы будем использовать оба датафрейма.

#### 2. Проверить данные на нормальность

Наша нулевая гипотеза будет утверждать, что данные распределены нормально.
Наша альтернативная гипотеза будет утверждать обратное.

Уровень значимости установим стандартный - 0.05

In [98]:
H0 = 'Данные распределены нормально'
Ha = 'Данные не распределены нормально'

alpha = 0.05

Протестируем оба датафрейма. Для проверки будем использовать тест Шапиро-Уилка.

In [99]:
def test_normal_distribution(df, df_name):
    print(f'Проверка нормального распределения для датафрейма "{df_name}"')

    _, p = shapiro(df)

    print(f'P-value = {p:.3f}')

    if p > alpha:
        print(H0)
    else:
        print(Ha)


test_normal_distribution(df_clean_avg, 'df_clean_avg')
print('')
test_normal_distribution(df_clean_med, 'df_clean_med')

Проверка нормального распределения для датафрейма "df_clean_avg"
P-value = 0.100
Данные распределены нормально

Проверка нормального распределения для датафрейма "df_clean_med"
P-value = 0.101
Данные распределены нормально


Результаты теста Шапиро-Уилка показывают, что нам не удалось опровергнуть нулевую гипотезу. Следовательно мы можем с большой долей вероятности полагать, что наши данные распределены нормально.

Отметим также, что тест вернул одинаковые результаты для обоих наших датафреймов.

#### 3. Выбрать тест на корреляцию и обосновать свой выбор

Какие существуют корреляционные тесты? В нашем арсенале их три:

- Корреляция Пирсона
- Корреляция Спирмена
- χ²-тест

Для выбора теста попробуем действовать метдом исключения.

**χ²-тест** нам не подходит, поскольку он применяется для категориальных переменных, а наши переменные являются числовыми и непрерывными.

**Корреляция Спирмена** нам не подходит, потому что она используется для поиска зависимостей между негаусовскими величинами, а наши данные, как мы выяснили, скорее всего имеют нормальное распределение.

Остается **параметрический тест на корреляцию Пирсона**. И действительно, он предназначен для поиска корреляции между непрерывными числовыми переменными, имеющими нормальное распределение.

#### 4. Проверить данные на наличие корреляции

Наша нулевая гипотеза будет предполагать, что корреляция в данных отсутствует. Альтернативная гипотеза будет предполагать, что корреляция в данных присутствует. 

Уровень значимости установим стандартный - 0.05

In [100]:
H0 = 'Корреляция в размерах мидий отстуствует'
Ha = 'Корреляция в размерах мидий присутствует'

alpha = 0.05

Протестируем оба датафрейма.

In [101]:
def test_pearson_corr(df, df_name):
    print(f'Проверка корреляции по методу Пирсона для датафрейма "{df_name}"')

    _, p = pearsonr(df['St.Petersburg'], df['Magadan'])

    print(f'P-value = {p:.3f}')

    if p > alpha:
        print(H0)
    else:
        print(Ha)
    
test_pearson_corr(df_clean_avg, 'df_clean_avg')
print('')
test_pearson_corr(df_clean_med, 'df_clean_med')

Проверка корреляции по методу Пирсона для датафрейма "df_clean_avg"
P-value = 0.559
Корреляция в размерах мидий отстуствует

Проверка корреляции по методу Пирсона для датафрейма "df_clean_med"
P-value = 0.539
Корреляция в размерах мидий отстуствует


#### 5. Сделать вывод по гипотезе

Тест Пирсона вернул значение **P-value** больше **α**. Следовательно, мы не можем опровергнуть нулевую гипотезу. Следовательно, с высокой долей вероятности мы можем утверждать, что размеры мидий из Санкт-Петербурга не оказывают влияния на размеры мидий из Магадана.

Нужно отметить, что тест Пирсона вернул одинаковые результаты для обоих наших датафреймов.


#### 6. Выбрать тест на сравнение и обосновать свой выбор.

Разобраться с лекционным материалом нелегко, многие вещи описаны довольно абстрактно. Ну что же, воспользуемся тем, что есть.

Поскольку наши данные имеют нормальное распределение, то мы будем выбирать тест на сравнение из числа **параметрических** статистических методов.

![Выбор статистических методов](images/dst-eda-4-7.png)

Сравнительные тесты ищут различия в средних параметрах по группам. Они помогают ответить на вопрос, влияет ли некая категория на среднее значение некоего числового признака. Например, влияет ли гендер на среднюю ЗП.

Причем категорий для сравнения может быть не две а больше, а сравниваемых групп может быть как несколько, так и всего одна.

Группы могут быть происходить из одной совокупности или из разных, где совокупность есть набор данных со схожими параметрами. Например если из учеников одного класса сформировать две группы, то эти группы будут происходить из одной совокупности.

Воспользуемся таблицей из лекционного материала.

![Параметрические тесты на сравнение выборок](images/dst-eda-4-9.png)

Чтобы выбрать правильный тест нам нужно понять сколько в нашем случае имеется независимых переменных, сколько зависимых, и каков характер сравниваемых групп.

У нас имеется одна независимая переменная — город производства. Зависимая переменная у нас тоже одна — средний размер мидий. Наши группы происходят из разных совокупностей поскольку мидии выращены в разных местах.

Согласно таблице мы можем воспользоваться использовать либо **независимый t-тест**, либо тест **ANOVA**.

Воспользуемся еще одной таблице из лекций.

![Подбор теста на сравнение выборок](images/dst-eda-4-10.png)

Согласно схеме выше, тест **ANOVA** применяют тогда, когда количество сравниваемых групп больше двух. Если же групп всего две, то применяют либо **T-тест**, либо **Z-тест**, что зависит от числа элементов в группах.

Поскольку в наших группах элементов меньше тридцати, то нам нужно использовать **T-тест**.

Получается, что согласно одной таблице нам нужно использовать **независимый t-тест**, а согласно другой — **T-тест**. Нет ли здесь противоречия?

# 🤔

Вероятно **T-тест**, упоминаемый в схеме выше, делает отсылку к *семейству T-тестов*. Похоже на то, ведь на странице модуля ScyPi, [посвященной статистическим тестам](https://docs.scipy.org/doc/scipy/reference/stats.html#module-scipy.stats), упоминается их четыре разновидности.

![T-тесты модуля ScyPi](images/Screenshot%202022-03-13%20at%2009.21.12.png)

Осталось выбрать один из них. Нам должен подойти **ttest_ind**, согласно описанию он делает как раз то, что нам нужно.

#### 7. Провести тест на сравнение

Декларируем наши гипотезы и выбираем уровень значимости.

In [102]:
H0 = 'Нет значимой разницы в средних значениях размеров мидий.'
Ha = 'Есть значимая разница в средних значениях размеров мидий.'
alpha = 0.05

Проводим тест для обоих наших датафреймов.

In [103]:
def run_ttest_ind(df, df_name):
    print(f'Независимый T-тест для датафрейма "{df_name}"')

    _, p = ttest_ind(df['St.Petersburg'], df['Magadan'])

    print(f'P-value = {p:.3f}')

    if p > alpha:
        print(H0)
    else:
        print(Ha)

run_ttest_ind(df_clean_avg, 'df_clean_avg')
print('')
run_ttest_ind(df_clean_med, 'df_clean_med')

Независимый T-тест для датафрейма "df_clean_avg"
P-value = 0.003
Есть значимая разница в средних значениях размеров мидий.

Независимый T-тест для датафрейма "df_clean_med"
P-value = 0.003
Есть значимая разница в средних значениях размеров мидий.


#### 8. Сделать вывод по гипотезе

Наш тест показал, что средний размер мидий, произведенных в Санкт-Петербурге и Магадане, существенно отличается. Но нужно отметить, что мы можем утверждать это лишь с вероятностью 95%, поскольку выбранный нами уровень значимости дает 5%-ный риск ошибки.

Отметим также, что тест вернул одинаковые результаты для обоих наших датафреймов.
